In [25]:
import praw
import os
from dotenv import load_dotenv
import json
from openai import OpenAI

In [26]:
# Load environment variables from .env file
load_dotenv()

# Reddit API credentials
client_id = os.getenv('REDDIT_CLIENT_ID')
client_secret = os.getenv('REDDIT_CLIENT_SECRET')
user_agent = os.getenv('REDDIT_USER_AGENT')
openai_key = os.getenv('OPENAI_API_KEY')
openai_client = OpenAI(api_key=openai_key)



In [16]:
# Initialize the Reddit instance
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

def fetch_comments(comment):
    """ Recursively fetch comments and their replies """
    comments_data = []
    comment_queue = comment.replies[:]  # Seed with top-level comments

    while comment_queue:
        current_comment = comment_queue.pop(0)
        comment_data = {
            "author": str(current_comment.author),
            "body": current_comment.body,
            "score": current_comment.score,
            "replies": fetch_comments(current_comment)
        }
        comments_data.append(comment_data)
    return comments_data

def fetch_reddit_topic(url):
    submission = reddit.submission(url=url)
    submission.comments.replace_more(limit=None)

    data = {
        "title": submission.title,
        "author": str(submission.author),
        "score": submission.score,
        "url": submission.url,
        "comments": []
    }

    for top_level_comment in submission.comments:
        comment_data = {
            "author": str(top_level_comment.author),
            "body": top_level_comment.body,
            "score": top_level_comment.score,
            "replies": fetch_comments(top_level_comment)
        }
        data["comments"].append(comment_data)

    return data

In [17]:
# Replace with your Reddit topic URL
reddit_topic_url = "https://www.reddit.com/r/GadgetsIndia/comments/1ekzay7/what_processor_does_your_phone_have_are_you/"
reddit_data = fetch_reddit_topic(reddit_topic_url)

final_data = json.dumps(reddit_data)

In [31]:
# Create the prompt
prompt = f"""
You are provided with a JSON dataset containing a discussion about mobile phone processors. Based on the data, summarize the following points:
1. What users like, prefer, or need as features in an application, focusing on problems that need to be solved, please output None if None.
2. What users don't like or hate, including things they find frustrating or want to avoid in an application, please output None if None.
3. Potential features that could be useful for the application, inferred from the discussion, please output None if None.

Discussion data:
{final_data}
"""

In [32]:
stream = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    stream=True,
)

# Print the summary as it streams
print("Summary:")
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

Summary:
Based on the discussion data provided, here's a summary of the users' feedback regarding mobile phone processors:

1. **Features Users Like/Need**:
   - Users appreciate efficient performance without lags.
   - Many mention the importance of battery efficiency, especially during high-intensity tasks, indicating a need for processors that do not overheat and drain battery rapidly.
   - Suggestions for checking performance through Geekbench scores or user reviews highlight a need for better ways to compare processors.

2. **Features Users Don’t Like/Hate**:
   - Users express frustration with overheating issues, particularly with processors such as Tensor and Exynos.
   - There's a dislike for lower-performing processors like Unisoc, with users calling them "Chinese trash" or "garbage" due to perceived lower quality and security vulnerabilities.
   - Many users mentioned their regrets in choosing Exynos processors, indicating dissatisfaction with their performance and reliabilit

KeyboardInterrupt: 